In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
# Define dict
songs = {}

In [8]:
# Billboard hot 100 page
url = "http://www.billboard.com/charts/hot-100"

In [9]:
def get_ranks(url, i):
    global songs
    # Get page
    r = requests.get(url)
    # Cook soup
    soup = BeautifulSoup(r.text, 'html.parser')
    # Extract ranks
    for song in soup.find_all(class_='chart-row__main-display'):
        song_rank   = song.find(class_='chart-row__current-week').string
        song_title  = song.find(class_='chart-row__song').string
        song_artist = song.find(class_='chart-row__artist').string.strip()
        song_name   = song_title + " : " + song_artist
        song_name   = str(song_name)
        if songs.get(song_name, 'hrily') is 'hrily':
            songs[song_name] = [[], []]
        songs[song_name][0].append(i)
        songs[song_name][1].append(int(song_rank))
    return soup

In [10]:
def print_ranks():
    global songs
    for name in songs.keys():
        print name + " = " + str(songs[name])

In [11]:
def get_previous_week_link(soup):
    link = soup.find(title="Previous Week")
    return link['href']

In [12]:
def get_last_n_ranks(n=1):
    url = "http://www.billboard.com/charts/hot-100"
    for i in range(n):
        soup = get_ranks(url, n-i)
        url  = "http://www.billboard.com" + get_previous_week_link(soup)

In [13]:
get_last_n_ranks(6)

In [14]:
print_ranks()

You're Welcome : Dwayne Johnson = [[1], [91]]
Love On The Brain : Rihanna = [[6, 5, 4, 3, 2, 1], [7, 8, 13, 14, 14, 17]]
Redbone : Childish Gambino = [[5, 4, 3, 2, 1], [99, 98, 94, 76, 63]]
Deja Vu : J. Cole = [[6, 5, 4, 3, 2, 1], [55, 50, 51, 47, 46, 48]]
Run Up : Major Lazer Featuring PARTYNEXTDOOR & Nicki Minaj = [[3], [66]]
Dirt On My Boots : Jon Pardi = [[6, 5, 4, 3, 2, 1], [42, 39, 46, 43, 44, 57]]
Heavy : Linkin Park Featuring Kiiara = [[6], [52]]
Million Reasons : Lady Gaga = [[6, 5, 4, 2], [30, 18, 4, 97]]
Starboy : The Weeknd Featuring Daft Punk = [[6, 5, 4, 3, 2, 1], [17, 14, 12, 7, 8, 5]]
Love Me Now : John Legend = [[6, 5, 4, 3, 2, 1], [69, 51, 45, 35, 33, 31]]
You Was Right : Lil Uzi Vert = [[1], [46]]
Halfway Off The Balcony : Big Sean = [[4], [74]]
Let Me Love You : DJ Snake Featuring Justin Bieber = [[6, 5, 4, 3, 2, 1], [31, 28, 20, 18, 16, 16]]
Sneakin' : Drake Featuring 21 Savage = [[2, 1], [95, 73]]
Not Nice : PARTYNEXTDOOR = [[6, 5, 4, 3, 2, 1], [88, 83, 82, 83, 83

In [16]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import math

def exponential_fit(x, a, b, c):
#     return a*x + b
    return a*x*x + b*x + c
#     return a*np.exp(-b*x) + c

nCorrect = 0
nCompute = 0
difference = 0
mDifference = 0

for name in songs.keys():
    if len(songs[name][0])<4:
        continue
    x = songs[name][0]
    y = songs[name][1]
#     print x, y
    x = x[1:]
    y = y[1:]
#     print x, y
    x = np.array(x)
    y = np.array(y)
    x = x[::-1]
    y = y[::-1]
#     print x, y
    try:
        fitting_parameters, covariance = curve_fit(exponential_fit, x, y)
        a, b, c = fitting_parameters

        next_y_correct = songs[name][1][0]

        next_x = 6
        next_y = exponential_fit(next_x, a, b, c)


        if next_y_correct == math.floor(next_y) or next_y_correct == math.floor(next_y+1):
            nCorrect = nCorrect + 1
        
        print name
        print y, next_y_correct, next_y
        
        nCompute = nCompute + 1
        difference = difference + abs(next_y_correct - next_y)
        if mDifference < abs(next_y_correct - next_y):
            mDifference = abs(next_y_correct - next_y)
    except:
        print "Prediction failed for "+name

print nCorrect, nCompute
print str( (1.0*nCorrect/nCompute)*100.0 ) + "%"
print 1.0*difference/nCompute
print mDifference

#     plt.plot(y)
#     plt.plot(np.append(y, next_y), 'ro')
#     plt.show()

Love On The Brain : Rihanna
[17 14 14 13  8] 7 4.99999999992
Redbone : Childish Gambino
[63 76 94 98] 99 101.05
Deja Vu : J. Cole
[48 46 47 51 50] 55 53.6
Dirt On My Boots : Jon Pardi
[57 44 43 46 39] 42 43.6
Million Reasons : Lady Gaga
[97  4 18] 30 72.3333333333
Starboy : The Weeknd Featuring Daft Punk
[ 5  8  7 12 14] 17 17.7999999999
Love Me Now : John Legend
[31 33 35 45 51] 69 62.6
Let Me Love You : DJ Snake Featuring Justin Bieber
[16 16 18 20 28] 31 36.0
Not Nice : PARTYNEXTDOOR
[95 83 83 82 83] 88 90.2000000001
Guys My Age : Hey Violet
[84 87 69 68] 83 55.5
No Heart : 21 Savage & Metro Boomin
[50 61 73 81 82] 85 82.6000000001
How Far I'll Go : Alessia Cara
[56 68 77 83 91] 94 95.0000000001
Better Man : Little Big Town
[41 35 34 40 35] 39 39.4
All Time Low : Jon Bellion
[22 19 20 24 26] 26 32.6
Don't Wanna Know : Maroon 5 Featuring Kendrick Lamar
[ 9  7  6  9 17] 19 27.0
Play That Song : Train
[69 60 41 59 59] 60 78.8000003537
Any Ol' Barstool : Jason Aldean
[93 95 84 76] 73 59